In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from joblib import dump

In [2]:
warnings.simplefilter("ignore")

In [3]:
df = pd.read_csv("LoanData.csv")

In [4]:
df1 =df.copy()


In [ ]:
df1.info()

# 1.Business Problem Understanding

It is a home loan elegibality prediction
The dataset has 12 columns which indicates the details of a users who are applied for loan
problem is check to basing on therir user provided details whether they are eligible or not

In [ ]:
df1

In [7]:
inp = df1.drop("Loan_Status",axis  = 1)
out = df1["Loan_Status"]

In [ ]:
inp,out

In [9]:
df1.drop(columns = "Loan_ID",inplace=True)

i checked all the column names they are all as of the syntax

# 2.Data Understanding

## Data Exploration

In [ ]:
df1

The given data set has 12 variables and  614 records

In [ ]:
df1.info()

In [ ]:
df1["Credit_History"].unique()

In [13]:
discrete_catogery = ["Gender","Married","Education","Self_Employed","Credit_History","Property_Area","Loan_Status"]
discrete_count = ["Dependents","Loan_Amount_Term"]
continous = ["ApplicantIncome","CoapplicantIncome","LoanAmount"]

In [ ]:
df1[continous+discrete_catogery+discrete_count]

In [ ]:
df1.isnull().sum(),df1.isnull().sum().sum()

In [ ]:
df1[discrete_catogery].describe()

In [ ]:
df1.dtypes

In [ ]:
df1["Self_Employed"].unique()

In [ ]:
df1["Dependents"].unique()

In [ ]:
df1[discrete_catogery].nunique()

In [ ]:
df1.isnull().sum()

In [ ]:
df1["Gender"].unique()

In [ ]:
df1["Married"].unique()

In [ ]:
df1["Married"].value_counts()

In [ ]:
df1["Loan_Status"].unique()

In [ ]:
df1["Property_Area"].unique()

### Exploratary Data Analysis 

In [ ]:
df1[discrete_catogery]

In [28]:
plt.rcdefaults()

In [ ]:
plt.figure(figsize=(18,8))
for i,j in zip(range(1,8),discrete_catogery):
    plt.title(j)
    plt.pie(x=df1[j].value_counts(),labels = df1[j].value_counts().index.tolist())
    plt.subplot(4,2,i)
plt.tight_layout()
plt.show()
    

In [ ]:
plt.pie(x=df1["Gender"].value_counts(),labels = df1["Gender"].value_counts().index.tolist())
plt.show()

In [31]:
plt.rcParams['figure.figsize'] = (10,5)

In [ ]:
plt.subplot(1,2,1)
sns.countplot(x=df1["Dependents"])
plt.subplot(1,2,2)
sns.countplot(x=df1["Loan_Amount_Term"])
plt.tight_layout()
plt.show()

In [ ]:
df1[continous]

In [ ]:
for i,j in zip(range(1,4),continous):
    plt.subplot(1,3,i)
    sns.histplot(df1[j])
    plt.gca().set_facecolor('pink')

plt.tight_layout()
plt.show()

In [ ]:
s1 =sns.pairplot(df1[continous])
s1.square_grid =True
plt.show()

In [ ]:
for i,j in zip(range(1,8),continous):
    plt.subplot(1,3,i)
    sns.boxplot(y=df1[j])
plt.tight_layout()
plt.show()

In [ ]:
sns.heatmap(df1[continous].corr(),annot =True)
plt.show()

# 3.Data Preparaion

Treating wrong data

In [ ]:
df1[discrete_catogery].isnull().sum()

In [ ]:
df1["Dependents"]

In [40]:
df1["Dependents"].replace({"3+":3},inplace=True)

In [41]:
df1["income"] = df1["ApplicantIncome"] + df1["CoapplicantIncome"]
df1.drop(columns=["ApplicantIncome","CoapplicantIncome"],inplace=True)

In [ ]:
df1

In [ ]:
df1["Gender"].isnull().sum()

In [44]:
df1["Gender"].fillna(df1["Gender"].mode()[0],inplace=True)

In [45]:
df1["Married"].fillna(df1["Married"].mode()[0],inplace=True)

In [46]:
df1["Self_Employed"].fillna(df1["Self_Employed"].mode()[0],inplace=True)

for continous and basing on the business problem it is not recommended to replace the on contious data and one column credicthistory

In [ ]:
df1[["LoanAmount","Loan_Amount_Term","income"]].isnull().sum()

In [48]:
df1.dropna(subset = ["LoanAmount","Loan_Amount_Term","Credit_History"],inplace=True)

In [ ]:
df1.isnull().sum()

In [50]:
df1["Dependents"].fillna(0,inplace=True)

In [ ]:
df1.isnull().sum()

In [ ]:
df1

In [ ]:
df1['Property_Area'].unique()

In [54]:
df1["Gender"].replace({"Male":1,"Female":0},inplace=True)
df1["Married"].replace({"Yes":1,"No":0},inplace=True)
df1["Education"].replace({"Graduate":1,"Not Graduate":0},inplace=True)
df1["Self_Employed"].replace({"Yes":1,"No":0},inplace=True)
df1["Property_Area"].replace({"Rural":0,"Semiurban":1,"Urban":2},inplace=True)
df1["Married"].replace({"Yes":1,"No":0},inplace=True)
df1["Loan_Status"].replace({"Y":1,"N":0},inplace=True)

In [ ]:
df1.dtypes

In [56]:
df1["Dependents"]=df1["Dependents"].astype('int') 

In [ ]:
df1["Dependents"].unique()

In [58]:
df1.to_csv("data.csv")

In [59]:
X = df1.drop("Loan_Status",axis=1)
y = df1["Loan_Status"]

In [60]:
 x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=93)

### Logestic Model

In [ ]:
model_lr  = LogisticRegression()
model_lr.fit(x_train,y_train)
print("Train :",model_lr.score(x_train,y_train))
print("Test :",model_lr.score(x_test,y_test))
print("CV :",cross_val_score(model_lr,x_train,y_train,scoring="accuracy",cv= 5).mean(),"\n\n")

### KNN model 

Finding best params

In [ ]:
e = KNeighborsClassifier()
params = {"n_neighbors":range(1,100)}
grid = GridSearchCV(e,param_grid=params,scoring="accuracy",cv = 5)
grid.fit(x_train,y_train)
print(grid.best_params_)
print("Best Estimator",grid.best_estimator_)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=13)
model_knn.fit(x_train,y_train)
print("Train :",model_knn.score(x_train,y_train))
print("Test :",model_knn.score(x_test,y_test))
print("CV :",cross_val_score(model_knn,x_train,y_train,scoring="accuracy",cv= 5).mean(),"\n\n")

### SVM model  

In [ ]:
e = SVC()
params = {"C":[0.01,0.1,1],"kernel":['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(e,param_grid=params,scoring="accuracy",cv = 5)
grid.fit(x_train,y_train)
print(grid.best_params_)
print("Best Estimator",grid.best_estimator_)

In [ ]:
model_svm = grid.best_estimator_
model_knn.fit(x_train,y_train)
print("Train :",model_knn.score(x_train,y_train))
print("Test :",model_knn.score(x_test,y_test))
print("CV :",cross_val_score(model_knn,x_train,y_train,scoring="accuracy",cv= 5).mean(),"\n\n")

In [ ]:
e = DecisionTreeClassifier()
params = {"max_depth":range(1,16),"criterion":["gini","entropy"]}
grid = GridSearchCV(e,param_grid=params,scoring="accuracy",cv = 5)
grid.fit(x_train,y_train)
print(grid.best_params_)
print("Best Estimator",grid.best_estimator_)

In [ ]:
model_dt = DecisionTreeClassifier(max_depth=1,random_state=93)
model_dt.fit(x_train,y_train)
print("Train :",model_dt.score(x_train,y_train))
print("Test :",model_dt.score(x_test,y_test))
print("CV :",cross_val_score(model_dt,x_train,y_train,scoring="accuracy",cv= 5).mean(),"\n\n")

In [68]:
a1  = grid.best_estimator_.feature_importances_

In [69]:
df2 = pd.DataFrame({"columns":X.columns},index=a1)

In [ ]:
df2[df2.index>=1]

In [ ]:
X_imp = X[['Credit_History']]
X_imp

In [72]:
x_train_dt,x_test_dt,y_train_dt,y_test_dt = train_test_split(X_imp,y,random_state=93)

In [ ]:
x_train_dt

In [ ]:
model_dt = DecisionTreeClassifier(max_depth=1,random_state=93)
model_dt.fit(x_train_dt,y_train_dt)
print("Train :",model_dt.score(x_train_dt,y_train_dt))
print("Test :",model_dt.score(x_test_dt,y_test_dt))
print("CV :",cross_val_score(model_dt,x_train,y_train,scoring="accuracy",cv= 5).mean(),"\n\n")

In [ ]:
dump(model_dt,"model.joblib")